In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
retail_data = pd.read_csv("/kaggle/input/retail-analysis-with-walmart-sales-data/WALMART_SALES_DATA.csv")

In [ ]:
# View the first 10 rows
retail_data.head(10)

In [ ]:
# Description of the data
retail_data.describe()

In [ ]:
retail_data.info()

the result above shows that there is no missinig data in the dataset

## **Question 1**
Which store has the maximum sale?

In [ ]:
# Row with the maximum sale
print(retail_data[retail_data.Weekly_Sales == retail_data.Weekly_Sales.max()])

Store 14 has the maximum sale

## **Question 2**
Which store has maximum standard deviation i.e., the sales vary a lot. Also, find out the coefficient of mean to standard deviation

In [ ]:
maxstd=pd.DataFrame(retail_data.groupby('Store').agg({'Weekly_Sales':['std','mean']}))

In [ ]:
#Just resetting the index.
maxstd = maxstd.reset_index()

In [ ]:
# coefficient of Variance = mean / standard Deviation
maxstd['COV'] = (maxstd[('Weekly_Sales','std')]/maxstd[('Weekly_Sales','mean')]) *100

In [ ]:
maxstd.loc[maxstd[('Weekly_Sales','std')]==maxstd[('Weekly_Sales','std')].max()]

Store 14 has the maximum standard deviation and its coefficient of Variation is 15.71

## **Question 3**
Which store has a good quarterly growth rate?

In [ ]:
# create a new column which shows the year and quarter
retail_data['quarter'] = pd.PeriodIndex(retail_data.Date, freq='Q')

In [ ]:
T2012Q2 = retail_data.loc[retail_data['quarter'] == "2012Q2", ["Weekly_Sales", 'Store']]

In [ ]:
T2012Q3 = retail_data.loc[retail_data['quarter'] == "2012Q3", ["Weekly_Sales", 'Store']]

In [ ]:
T2012Q2_sum_per_store = pd.DataFrame(T2012Q2.groupby('Store')['Weekly_Sales'].sum())
T2012Q2_sum_per_store.reset_index(inplace=True)
T2012Q3_sum_per_store = pd.DataFrame(T2012Q3.groupby('Store')['Weekly_Sales'].sum())
T2012Q3_sum_per_store.reset_index(inplace=True)

In [ ]:
T2012Q2_sum_per_store['Weekly_Sales_Q3'] = T2012Q3_sum_per_store['Weekly_Sales']

In [ ]:
T2012Q2_sum_per_store['Growth Rate'] = ((T2012Q2_sum_per_store.Weekly_Sales_Q3 - T2012Q2_sum_per_store.Weekly_Sales)/T2012Q2_sum_per_store.Weekly_Sales)*100

In [ ]:
T2012Q2_sum_per_store.loc[T2012Q2_sum_per_store['Growth Rate']==T2012Q2_sum_per_store['Growth Rate'].max()]

The Store with good quarterly growth rate is Store 16

## **Question 4**
Find out the holiday that has the higher sales than the mean sales in non-holiday season ll together

In [ ]:
import datetime as dt
retail_data.Date = pd.to_datetime(retail_data.Date)

In [ ]:
Holiday_Week = retail_data.loc[retail_data['Holiday_Flag']==1]


In [ ]:
from matplotlib import pyplot as plt
import time
retail_data['Date'] = pd.to_datetime(retail_data.Date)

In [ ]:
def plot_line(df, holiday_dates, holiday_label):
    fig, ax = plt.subplots(figsize=(15,5))
    ax.plot(df['Date'], df['Weekly_Sales'], label=holiday_label)
    
    for day in holiday_dates:
        day = dt.datetime.strptime(day, '%d-%m-%Y').date()
        plt.axvline(x=day, linestyle='--', c='r')
        
    plt.title(holiday_label)
    x_dates = df['Date'].dt.strftime('%Y-%m-%d').sort_values().unique()
    plt.show()

In [ ]:
total_sales = retail_data.groupby('Date')['Weekly_Sales'].sum().reset_index()
Super_Bowl=['12-2-2010', '11-2-2011', '10-2-2012']
Labor_Day = ['10-9-2010', '9-9-2011', '7-9-2012']
Thanksgiving = ['26-11-2010', '25-11-2011', '23-11-2012']
Christmas = ['31-12-2010', '30-12-2011', '28-12-2012']

In [ ]:
plot_line(total_sales, Super_Bowl, 'Super Bowl')

In [ ]:
plot_line(total_sales, Labor_Day, 'Labour Day')

In [ ]:
plot_line(total_sales, Thanksgiving, 'Thanksgiving Week')

In [ ]:
plot_line(total_sales, Christmas, 'Christmas')

In [ ]:
from matplotlib import pyplot as plt
pd.DatetimeIndex(retail_data['Date'])
monthly = retail_data.groupby(pd.Grouper(key='Date', freq='1M')).sum()# groupby each 1 month
monthly=monthly.reset_index()
fig, ax = plt.subplots(figsize=(10,6))
X = monthly['Date']
Y = monthly['Weekly_Sales']
plt.plot(X,Y)
plt.title('Month Wise Sales')
plt.xlabel('Months')
plt.ylabel('Weekly_Sales')

In [ ]:
quarterly = retail_data.groupby(pd.Grouper(key='Date', freq='3M')).sum()# groupby each 3e month
quarterly=quarterly.reset_index()
fig, ax = plt.subplots(figsize=(10,6))
X = quarterly['Date']
Y = quarterly['Weekly_Sales']
plt.plot(X,Y)
plt.title('Quarterly Sales')
plt.xlabel('Months')
plt.ylabel('Weekly_Sales')

In [ ]:
semesterly = retail_data.groupby(pd.Grouper(key='Date', freq='3M')).sum()# groupby each 3e month
semesterly=semesterly.reset_index()
fig, ax = plt.subplots(figsize=(10,6))
X = semesterly['Date']
Y = semesterly['Weekly_Sales']
plt.plot(X,Y)
plt.title('Semester Sales')
plt.xlabel('Years')
plt.ylabel('Weekly_Sales')

In [ ]:
# A little feature engineering
import seaborn as sns
corr = retail_data.corr()

plt.figure(figsize=(8, 5))
sns.heatmap(corr, annot=True)
plt.show()

In [ ]:
retail_data['Day']=retail_data['Date'].dt.day
retail_data = retail_data.drop('Date',axis=1)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import confusion_matrix, r2_score

In [ ]:
df_v2 = pd.get_dummies(retail_data, columns = ['Holiday_Flag','Store'])
y = df_v2['Weekly_Sales']
X= df_v2.drop(['Weekly_Sales', 'quarter'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
ln_model = LinearRegression()
ln_model.fit(X_train, y_train)

In [ ]:
y_pred = ln_model.predict(X_test)
print("r2 score:", r2_score(y_test,y_pred))